In [1]:
from bs4 import BeautifulSoup
import json # library to handle JSON files
import numpy as np
import pandas as pd
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import urllib
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

# First we'll use Beautiful soup to parse the wikipedia page

In [2]:
pd.options.display.max_rows = 4000
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = urllib.request.urlopen(url)
rawhtml = response.read()
soup = BeautifulSoup(rawhtml, 'html.parser')

In [3]:

pclist = []
for table_row in soup.select("table.wikitable tr"):
    cells = table_row.findAll('td')
    #row = [table_row.text.strip() for table_row in cells]
    #pclist.append(row)
    #note: do not include "Not assigned" values or grayed out values
    if len(cells) > 0:
        pc = cells[0].text.strip()
        b = cells[1].text.strip()
        n = cells[2].text.strip()

        if (b == "Not assigned" and n=="Not assigned"):
           #skip row
            a="skipping"
            #print('skipping')
       # elif (b == "Not assigned"):
       #     pclist.append([pc,n,n])
        elif(n=="Not assigned"):
            #if the neighbourhood is not assigned, then subsitute in the borough
            pclist.append([pc,b,b])
        else:
            pclist.append([pc,b,n])
            

toronto_df = pd.DataFrame(pclist, columns=["FSA", "Borough", "Neighborhood"])
toronto_df = toronto_df.sort_values(by=['FSA'])
toronto_df.head(15)

,FSA,Borough,Neighborhood
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
22,M1C,Scarborough,Port Union
21,M1C,Scarborough,Rouge Hill
20,M1C,Scarborough,Highland Creek
32,M1E,Scarborough,Guildwood
33,M1E,Scarborough,Morningside
34,M1E,Scarborough,West Hill
38,M1G,Scarborough,Woburn
42,M1H,Scarborough,Cedarbrae


In [4]:
toronto_df.shape

(210, 3)

# create a dataframe out of the retrieved info

In [5]:
pclist = []
prevPC = "" # stores the previous postal code to see if we need to append neighbourhoods or create a new row
currNeighbourhoods = ""
prevBorough = ""
for table_row in soup.select("table.wikitable tr"):
    cells = table_row.findAll('td')
    if len(cells) > 0:
        pc = cells[0].text.strip()
        b = cells[1].text.strip()
        n = cells[2].text.strip()
        if ((b == "Not assigned" and n=="Not assigned") or b == "blank"):
           #skip row
            a="skipping"
        else:
            if (n=="Not assigned"):
                n = b; # set the neighbourhood equal to the borough
            
            if (pc == prevPC):
                # append neighbourhoods into comma delimited string
                currNeighbourhoods = currNeighbourhoods + ", " + n
                 
            else:
                # new postal code, so write out previous row
                if (prevPC != ""):
                    pclist.append([prevPC,prevBorough,currNeighbourhoods])
                prevPC = pc # set a new PrevPC value for next loop
                prevBorough = b
                currNeighbourhoods = n # start a new neighbourhoods string
# write out last row
pclist.append([prevPC,prevBorough,currNeighbourhoods])               
            
toronto_df = pd.DataFrame(pclist, columns=["PostalCode", "Borough", "Neighborhood"])
toronto_df = toronto_df.sort_values(by=['PostalCode'])#.reset_index()
toronto_df#.head(15)

,PostalCode,Borough,Neighborhood
6,M1B,Scarborough,"Rouge, Malvern"
12,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae
32,M1J,Scarborough,Scarborough Village
38,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
44,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
51,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
58,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
toronto_df.shape

(103, 3)